In [1]:
# import necessary packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.metrics import auc

In [2]:
def gaussian_mle (trainfile,a,b):
    fileData = open(trainfile, "r",encoding='utf-8-sig')
    lines = fileData.readlines()
    data = []
    for line in lines:
        x = line.split()
        data.append(x)
    
    df = pd.DataFrame(data,index = None, columns = list(range (len(data[0]))) )
    df = df.apply(pd.to_numeric)
    col = len (df.columns)-1
    X = df.iloc[:,0:col].values
    Y = df.iloc[:,col].values
        
    class0 = df.loc[(df[col] == 0)]
    class1 = df.loc[(df[col] == 1)]

    col0 = len (class0.columns)-1
    X0 = class0.iloc[:,0:col0].values
    mat0 = np.matrix(X0)    

    col1 = len (class1.columns)-1
    X1 = class1.iloc[:,0:col1].values
    mat1 = np.matrix(X1)  

    class_type = np.unique(Y)
    n_classes = class_type.shape[0]
    mean_vectors = []
    for cls in class_type:
        mean_vectors.append(np.mean(X[Y==cls], axis=0))
    
    mu0 = mean_vectors[0]
    mu1 = mean_vectors[1]
    cov0 = ((mat0 - mu0).T.dot((mat0 - mu0)))/mat0.shape[0]
    cov1 = ((mat1 - mu1).T.dot((mat1 - mu1)))/mat1.shape[0]

    # find covariance matrices for CASE I
    I = np.identity(col)
    sigma3 = cov0[a,b]*I

    cov_vectors = [cov0,cov1,sigma3] 
    dim_data = df.shape[0]
    dim_class0 = class0.shape[0]
    dim_class1 = class1.shape[0]

    return mean_vectors, cov_vectors, dim_data, dim_class0,dim_class1

# gaussian_mle ('nX_PIMA_TR.txt',cov0[4,4])

In [3]:
# calculate for different probabilities and create dictionary with probability as keys and accuracy as values
def mpp(testFile,mean1,mean2,detClass1,detClass2,invClass1,invClass2,probClass1):
    probClass2 = 1-probClass1
    
    # read the test data file
    fileData1 = open(testFile, "r",encoding='utf-8-sig')
    lines = fileData1.readlines()
    
    data = []
    for line in lines:
        x = line.split()
        data.append(x)
    
    df = pd.DataFrame(data,index = None, columns = list(range (len(data[0]))) )
    df = df.apply(pd.to_numeric)
    col = len (df.columns)
    X = df.iloc[:,0:col].values

    # create a matrix from test data for calculation
    testSet = np.matrix(X)

    n = testSet.shape[1]
    
    predictedMatrix = np.zeros((testSet.shape[0], 2))

    correctGuesses = 0
    line1 = np.zeros((1, n-1))
    for i in range(testSet.shape[0]):
        for j in range (n-1):
            line1[:,j] = testSet[i,j]

            line2 = line1.T
            mahalanobis = line2 - mean1
            mahalanobis1 = mahalanobis.T
            mahalanobis2 = mahalanobis1 * invClass1
            mahalanobis3 = mahalanobis2 * (line2 - mean1)
            varMahalanobis = float(-0.5 * mahalanobis3[0][0])
            probIsClass1 = float((1.0 / math.sqrt(2 * math.pi * detClass1)) * math.exp(varMahalanobis) * (probClass1))

            mahalanobiss = line2 - mean2
            mahalanobiss1 = mahalanobiss.T
            mahalanobiss2 = mahalanobiss1 * invClass2
            mahalanobiss3 = mahalanobiss2 * (line2 - mean2)
            varMahalanobiss = float(-0.5 * mahalanobiss3[0][0])

            probIsClass2 = float((1.0 / math.sqrt(2 * math.pi * detClass2)) * math.exp(varMahalanobiss) * (probClass2))

            predictedClass = 0
            if probIsClass2 > probIsClass1:
                predictedClass = 1

            if predictedClass == testSet[i,n-1]: 
                correctGuesses+=1

            error = float(min(probIsClass2, probIsClass1))

            predictedMatrix[i][1] = error
            predictedMatrix[i][0] = predictedClass

    return testSet,predictedMatrix


In [4]:
#calculate TP,TN,FP,FN values
def performance_measure(testSet, predictedMatrix):
    n = testSet.shape[1]
    true_class = []
    predicted_class = []
    for i in range(testSet.shape[0]):
        true = testSet[i,n-1]
        true_class.append(true)
        
    for i in range(predictedMatrix.shape[0]):
        predicted = predictedMatrix[i,0]
        predicted_class.append(predicted)
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    posneg = []
    for i in range(len(predicted_class)): 
        if true_class[i]==predicted_class[i]==1:
            TP += 1
        if predicted_class[i]==1 and true_class[i]!=predicted_class[i]:
            FP += 1
        if true_class[i]==predicted_class[i]==0:
            TN += 1
        if predicted_class[i]==0 and true_class[i]!=predicted_class[i]:
            FN += 1

    posneg.append(TP)
    posneg.append(FP)
    posneg.append(TN)
    posneg.append(FN)
    
    return posneg


In [5]:
#calculate sensitivity,specificity, TPR, FPR
def accuracy(performance_list):
    sensitivity = performance_list[0]/(performance_list[0]+performance_list[3])
    specificity = performance_list[2]/(performance_list[2]+performance_list[1])
    TPR = sensitivity
    FPR = 1 - specificity
    roc = [sensitivity,specificity, TPR, FPR]
    return roc


In [6]:
#calculate accuracy (the probability of a correct decision)
def getAccuracy(performance_list):
    acc = (performance_list[0]+performance_list[2])/sum(performance_list)
    return acc

In [7]:
#calculate sensitivity,specificity, TPR, FPR for different prior probabilities
def perf_list (filename,mean1,mean2,detClass1,detClass2,invClass1,invClass2):
    probClass1 = []
    x = 0
    while x<1:
        x+=.005
        probClass1.append(x)

    perf = []
    for p in probClass1:
        testSet,predictedMatrix= mpp(filename,mean1,mean2,detClass1,detClass2,invClass1,invClass2,p)
        performance = performance_measure(testSet, predictedMatrix)
        acc = accuracy(performance)
        perf.append(acc)
    return perf


In [8]:
#plot ROC curve and calculate the area under the curve
def plot_roc(perf_list):
    plt.figure(figsize=(6,5))
    lw = 2

    TPR = []
    FPR = []
    for i in range(len(perf_list)):
        x = perf_list[i][2]
        TPR.append(x)
        y = perf_list[i][3]
        FPR.append(y)
        
    roc_auc = auc(FPR, TPR) 

    plt.plot(FPR, TPR, color='darkorange',lw=2,label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    
    # plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic curve')
    plt.legend(loc="lower right")
    plt.show()


In [9]:
#plot sensitivity and specificity curve against different priors
def plot_sens_spec(perf_list):
    plt.figure(figsize=(6,5))
    lw = 2
    probClass1 = []
    x = 0
    while x<1:
        x+=.005
        probClass1.append(x)

    sensitivity = []
    specificity = []
    for i in range(len(perf_list)):
        x = perf_list[i][0]
        sensitivity.append(x)
        y = perf_list[i][1]
        specificity.append(y)
        
    line1 = plt.plot(probClass1, sensitivity,'darkorange',lw = 2,label ='sensitivity')
    line2 = plt.plot(probClass1, specificity,'b',lw = 2,label ='specificity')

    plt.xlabel('Prior Probabilities')
#     plt.ylabel('Accuracy')
    plt.legend(loc="lower right")
    plt.title('Sensitivity & Specificity Curve for Different Prior Probabilities')
    plt.show() 

In [10]:
#plot TP,TN,FP,FN values for all FOUR classifiers
def plot_acc(knn, case1, case2, case3):        
    # plot accuracy with respect to 4 classifiers
    plt.clf()
    fig, axs = plt.subplots(1,1,figsize=(6,5))
    opacity = .7
    w = .25
    x = ['TP', 'FP', 'TN', 'FN']
    
    a = [x for x, _ in enumerate(knn)]
    b = [x + w for x in a]
    c = [x + w for x in b]
    d = [x + w for x in c]

    bar1 = axs.bar(a, knn, color='b', width=w, edgecolor='white', label='KNN')
    bar2 = axs.bar(b, case1, color='g', width=w, edgecolor='white', label='CaseI')
    bar3 = axs.bar(c, case2, color='darkorange', width=w, edgecolor='white', label='CaseII')
    bar4 = axs.bar(d, case3, color='r', width=w, edgecolor='white', label='CaseIII')
    

    # access the bar attributes to place the text in the appropriate location
    for bar in bar1:
        yval = bar.get_height()
        plt.text(bar.get_x(), yval + .5, yval)
    for bar in bar2:
        yval = bar.get_height()
        plt.text(bar.get_x(), yval + .5, yval)
    for bar in bar3:
        yval = bar.get_height()
        plt.text(bar.get_x(), yval + .5, yval)
    for bar in bar4:
        yval = bar.get_height()
        plt.text(bar.get_x(), yval + .5, yval)

    plt.xticks([x + 1.5*w for x in range(len(x))], x)
    plt.legend(loc='upper left')
    plt.ylabel('Accuracy')
    plt.title('Performance Analysis of Four Classifiers')
    plt.tight_layout()
    plt.show()

In [11]:
#calculate sensitivity and specificity curve against different eigenvalues in PCA
def vary_eigVal(trainfile,testfile):
    # uncomment this line for output
    mean_vectors, cov_vectors,dim_data, dim_class0,dim_class1  = gaussian_mle (trainfile,0,0)
    mean1 = np.matrix(mean_vectors[0]).T
    mean2 = np.matrix(mean_vectors[1]).T
    cov0 = cov_vectors[0]
    cov1 = cov_vectors[1]
    cov2 = cov_vectors [2]
    # calculate determinant and inverse of new covariance Matrix (case 1: covariance matrices are equal to (sigma^2)I)
    detClass1 = np.linalg.det(cov2)
    detClass2 = detClass1
    invClass1 = np.linalg.inv(cov2)
    invClass2 = invClass1
    # calculate determinant and inverse of new covariance matrices (case 2: covariance matrices is equal)
    detClass12 = np.linalg.det(cov1)
    detClass22 = detClass12
    invClass12 = np.linalg.inv(cov1)
    invClass22 = invClass12
    # calculate determinant and inverse of new covariance matrices (case 3: covariance matrices are different)
    detClass13 = np.linalg.det(cov0)
    invClass13 = np.linalg.inv(cov0)
    invClass23 = np.linalg.inv(cov1)
    detClass23 = np.linalg.det(cov1)

    p = dim_class0/dim_data
    
    testSet1,predictedMatrix1= mpp(testfile,mean1,mean2,detClass1,detClass2,invClass1,invClass2,p)
    performance1 = performance_measure(testSet1, predictedMatrix1)
    case1 = accuracy(performance1)

    testSet2,predictedMatrix2= mpp(testfile,mean1,mean2,detClass12,detClass22,invClass12,invClass22,p)
    performance2 = performance_measure(testSet2, predictedMatrix2)
    case2 = accuracy(performance2)
    
    testSet3,predictedMatrix3= mpp(testfile,mean1,mean2,detClass13,detClass23,invClass13,invClass23,p)
    performance3 = performance_measure(testSet3, predictedMatrix3)
    case3 = accuracy(performance3)

    return case1,case2,case3

case1p1,case2p1,case3p1 = vary_eigVal('pX_PIMA_TR_0.01.txt','pX_PIMA_TE_0.01.txt')
case1p2,case2p2,case3p2 = vary_eigVal('pX_PIMA_TR_0.05.txt','pX_PIMA_TE_0.05.txt')
case1p3,case2p3,case3p3 = vary_eigVal('pX_PIMA_TR_0.1.txt','pX_PIMA_TE_0.1.txt')
case1p4,case2p4,case3p4 = vary_eigVal('pX_PIMA_TR_0.2.txt','pX_PIMA_TE_0.2.txt')
case1p5,case2p5,case3p5 = vary_eigVal('pX_PIMA_TR_0.4.txt','pX_PIMA_TE_0.4.txt')
case1p6,case2p6,case3p6 = vary_eigVal('pX_PIMA_TR_0.5.txt','pX_PIMA_TE_0.5.txt')
case1p7,case2p7,case3p7 = vary_eigVal('pX_PIMA_TR_0.7.txt','pX_PIMA_TE_0.7.txt')


In [12]:
#plot sensitivity and specificity curve against different eigenvalues in PCA
def plot_vary_eigVal(p1,p2,p3,p4,p5,p6,p7):
    e = [x for x in range(1,8)]
    sensitivity = [p7[0],p6[0],p5[0],p4[0],p3[0],p2[0],p1[0]]
    specificity = [p7[1],p6[1],p5[1],p4[1],p3[1],p2[1],p1[1]]
    
    line1 = plt.plot(e, sensitivity,'darkorange',lw = 2,label ='sensitivity')
    line2 = plt.plot(e, specificity,'b',lw = 2,label ='specificity')

    plt.xlabel('Number of Eigenvalues')
#     plt.ylabel('Accuracy')
    plt.legend(loc="lower right")
    plt.title('Sensitivity & Specificity Curve for Different Eigenvalues (Discriminant Analysis)')
    plt.show() 

parameter calculation for normalized datset

In [13]:
# uncomment this line for output
mean_vectors, cov_vectors,dim_data, dim_class0,dim_class1  = gaussian_mle ('nX_PIMA_TR.txt',4,4)
mean1 = np.matrix(mean_vectors[0]).T
mean2 = np.matrix(mean_vectors[1]).T
cov0 = cov_vectors[0]
cov1 = cov_vectors[1]
cov2 = cov_vectors [2]
# calculate determinant and inverse of new covariance Matrix (case 1: covariance matrices are equal to (sigma^2)I)
detClass1 = np.linalg.det(cov2)
detClass2 = detClass1
invClass1 = np.linalg.inv(cov2)
invClass2 = invClass1


In [14]:
# calculate determinant and inverse of new covariance matrices (case 2: covariance matrices is equal)
detClass12 = np.linalg.det(cov1)
detClass22 = detClass12
invClass12 = np.linalg.inv(cov1)
invClass22 = invClass12


In [15]:
# calculate determinant and inverse of new covariance matrices (case 3: covariance matrices are different)
detClass13 = np.linalg.det(cov0)
invClass13 = np.linalg.inv(cov0)
invClass23 = np.linalg.inv(cov1)
detClass23 = np.linalg.det(cov1)


In [16]:
testSet,predictedMatrix = mpp("nX_PIMA_TE.txt",mean1,mean2,detClass1,detClass2,invClass1,invClass2,dim_class0/dim_data)
case1 = performance_measure(testSet, predictedMatrix)
testSet2,predictedMatrix2 = mpp("nX_PIMA_TE.txt", mean1,mean2,detClass12,detClass22,invClass12,invClass22,dim_class0/dim_data)
case2 = performance_measure(testSet2, predictedMatrix2)
testSet3,predictedMatrix3 = mpp("nX_PIMA_TE.txt",mean1,mean2,detClass13,detClass23,invClass13,invClass23,dim_class0/dim_data)
case3 = performance_measure(testSet3, predictedMatrix3)
acc1 = accuracy(case1)
acc2 = accuracy(case2)
acc3 = accuracy(case3)
# print (case1,case2,case3)
# print (acc1, acc2, acc3)
accN = [getAccuracy(case1),getAccuracy(case2),getAccuracy(case3)]
# print (accN)

In [17]:
da_case1n=perf_list ("nX_PIMA_TE.txt",mean1,mean2,detClass1,detClass2,invClass1,invClass2)
da_case2n=perf_list ("nX_PIMA_TE.txt",mean1,mean2,detClass12,detClass22,invClass12,invClass22)
da_case3n=perf_list ("nX_PIMA_TE.txt",mean1,mean2,detClass13,detClass23,invClass13,invClass23)

parameter calculation for PCA datset

In [18]:
mean_vectorsP, cov_vectorsP,dim_dataP, dim_class0P,dim_class1P  = gaussian_mle ('pX_PIMA_TR.txt',4,4)
mean1P = np.matrix(mean_vectorsP[0]).T
# print (mean1)
mean2P = np.matrix(mean_vectorsP[1]).T
cov0P = cov_vectorsP[0]
cov1P = cov_vectorsP[1]
cov2P = cov_vectorsP[2]
detClass1P1 = np.linalg.det(cov2P)
detClass2P1 = detClass1P1
invClass1P1 = np.linalg.inv(cov2P)
invClass2P1 = invClass1P1

In [19]:
detClass1P2 = np.linalg.det(cov0P)
detClass2P2 = detClass1P2
invClass1P2 = np.linalg.inv(cov0P)
invClass2P2 = invClass1P2

In [20]:
detClass1P3 = np.linalg.det(cov0P)
invClass1P3 = np.linalg.inv(cov0P)
invClass2P3 = np.linalg.inv(cov1P)
detClass2P3 = np.linalg.det(cov1P)

In [21]:
testSetP,predictedMatrixP = mpp("pX_PIMA_TE.txt",mean1P,mean2P,detClass1P1,detClass2P1,invClass1P1,invClass2P1,dim_class0P/dim_dataP)
case1P = performance_measure(testSetP, predictedMatrixP)
testSet2P,predictedMatrix2P = mpp("pX_PIMA_TE.txt", mean1P,mean2P,detClass1P2,detClass2P2,invClass1P2,invClass2P2,dim_class0P/dim_dataP)
case2P = performance_measure(testSet2P, predictedMatrix2P)
testSet3P,predictedMatrix3P = mpp("pX_PIMA_TE.txt",mean1P,mean2P,detClass1P3,detClass2P3,invClass1P3,invClass2P3,dim_class0P/dim_dataP)
case3P = performance_measure(testSet3P, predictedMatrix3P)
acc1p = accuracy(case1P)
acc2p = accuracy(case2P)
acc3p = accuracy(case3P)

# print (case1P,case2P,case3P)
# print (acc1p, acc2p, acc3p)

accP = [getAccuracy(case1P),getAccuracy(case2P),getAccuracy(case3P)]
# print (accP)

In [22]:
da_case1p=perf_list ("pX_PIMA_TE.txt",mean1P,mean2P,detClass1P1,detClass2P1,invClass1P1,invClass2P1)
da_case2p=perf_list ("pX_PIMA_TE.txt",mean1P,mean2P,detClass1P2,detClass2P2,invClass1P2,invClass2P2)
da_case3p=perf_list ("pX_PIMA_TE.txt",mean1P,mean2P,detClass1P3,detClass2P3,invClass1P3,invClass2P3)

parameter calculation for FLD datset

In [23]:
mean_vectorsF, cov_vectorsF,dim_dataF, dim_class0F,dim_class1F  = gaussian_mle ('fX_PIMA_TR.txt',0,0)
mean1F = np.matrix(mean_vectorsF[0]).T
mean2F = np.matrix(mean_vectorsF[1]).T
cov0F = cov_vectorsF[0]
cov1F = cov_vectorsF[1]
cov2F = cov_vectorsF[2]
detClass1F1 = np.linalg.det(cov2F)
detClass2F1 = detClass1F1
invClass1F1 = np.linalg.inv(cov2F)
invClass2F1 = invClass1F1


In [24]:
detClass1F2 = np.linalg.det(cov0F)
detClass2F2 = detClass1F2
invClass1F2 = np.linalg.inv(cov0F)
invClass2F2 = invClass1F2


In [25]:
detClass1F3 = np.linalg.det(cov0F)
invClass1F3 = np.linalg.inv(cov0F)
invClass2F3 = np.linalg.inv(cov1F)
detClass2F3 = np.linalg.det(cov1F)

In [50]:
testSetF,predictedMatrixF = mpp("fX_PIMA_TE.txt",mean1F,mean2F,detClass1F1,detClass2F1,invClass1F1,invClass2F1,dim_class0F/dim_dataF)
case1F = performance_measure(testSetF, predictedMatrixF)
testSet2F,predictedMatrix2F = mpp("fX_PIMA_TE.txt", mean1F,mean2F,detClass1F2,detClass2F2,invClass1F2,invClass2F2,dim_class0F/dim_dataF)
case2F = performance_measure(testSet2F, predictedMatrix2F)
testSet3F,predictedMatrix3F = mpp("fX_PIMA_TE.txt",mean1F,mean2F,detClass1F3,detClass2F3,invClass1F3,invClass2F3,dim_class0F/dim_dataF)
case3F = performance_measure(testSet3F, predictedMatrix3F)
acc1f = accuracy(case1F)
acc2f = accuracy(case2F)
acc3f = accuracy(case3F)

# print (case1F,case2F,case3F)
# print (acc1f, acc2f, acc3f)
accF = [getAccuracy(case1F),getAccuracy(case2F),getAccuracy(case3F)]
# print (accF)

In [27]:
da_case1f=perf_list ("fX_PIMA_TE.txt",mean1F,mean2F,detClass1F1,detClass2F1,invClass1F1,invClass2F1)
da_case2f=perf_list ("fX_PIMA_TE.txt",mean1F,mean2F,detClass1F2,detClass2F2,invClass1F2,invClass2F2)
da_case3f=perf_list ("fX_PIMA_TE.txt",mean1F,mean2F,detClass1F3,detClass2F3,invClass1F3,invClass2F3)

output cells

uncomment for output

In [28]:
# x = plot_roc (da_case1n)

In [29]:
# y = plot_roc (da_case2n)

In [30]:
# plot_roc (da_case3n)

In [31]:
# plot_roc (da_case1p)

In [32]:
# plot_roc (da_case2p)

In [33]:
# plot_roc (da_case3p)

In [57]:
# plot_roc (da_case1f)

In [58]:
# plot_roc (da_case2f)

In [55]:
# plot_roc (da_case3f)

In [54]:
# knn = [56, 23, 200, 53]
# knnp = [60, 28, 195, 49]
# knnf = [66, 28, 195, 43]

# plot_acc (knn, case1, case2, case3)
# plot_acc (knnp, case1P, case2P, case3P)
# plot_acc (knnf, case1F, case2F, case3F)


In [38]:
# plot_sens_spec(da_case1n)

In [39]:
# plot_sens_spec(da_case2n)

In [40]:
# plot_sens_spec(da_case3n)

In [41]:
# plot_sens_spec(da_case1p)

In [42]:
# plot_sens_spec(da_case2p)

In [43]:
# plot_sens_spec(da_case3p)

In [53]:
# plot_sens_spec(da_case1f)

In [52]:
# plot_sens_spec(da_case2f)

In [51]:
# plot_sens_spec(da_case3f)

In [47]:
# plot_vary_eigVal(case1p1,case1p2,case1p3,case1p4,case1p5,case1p6,case1p7)

In [48]:
# plot_vary_eigVal(case2p1,case2p2,case2p3,case2p4,case2p5,case2p6,case2p7)

In [49]:
# plot_vary_eigVal(case3p1,case3p2,case3p3,case3p4,case3p5,case3p6,case3p7)